<a href="https://colab.research.google.com/github/HwayoungYoon/coin-analysis/blob/main/%ED%8A%B8%EC%9C%84%ED%84%B0_%ED%81%AC%EB%A1%A4%EB%A7%81_selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.l

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import requests
from selenium.webdriver.common.desired_capabilities import  DesiredCapabilities
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import datetime as dt
import pandas as pd

In [ ]:
#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.

options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)



---



https://jhnoru.tistory.com/59

In [ ]:
def init_driver():
  
  # initiate the driver:
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')        # Head-less 설정
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver', options=options)
  
  # set a default wait time for the browser [5 seconds here]:
  driver.wait = WebDriverWait(driver, 5)
  
  return driver

In [ ]:
def close_driver(driver):
  driver.close()
  return

In [ ]:
class wait_for_more_than_n_elements_to_be_present(object):
  def __init__(self, locator, count):
    self.locator = locator
    self.count = count
  def __call__(self, driver):
    try:
      elements = EC._find_elements(driver, self.locator)
      return len(elements) > self.count
    except StaleElementReferenceException:
      return False

In [ ]:
def search_twitter(driver, query):
  
  # wait until the search box has loaded:
  box = driver.wait.until(EC.presence_of_element_located((By.NAME, "q")))
  
  # find the search box in the html:
  driver.find_element_by_name("q").clear()
  
  # enter your search string in the search box:
  box.send_keys(query)
  
  # submit the query (like hitting return):
  box.submit()
  
  # initial wait for the search results to load
  wait = WebDriverWait(driver, 10)
  
  try:
    # wait until the first search result is found. Search results will be tweets, which are html list items and have the class='data-item-id':
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "li[data-item-id]")))
    
    # scroll down to the last tweet until there are no more tweets:
    while True:
      
      # extract all the tweets:
      tweets = driver.find_elements_by_css_selector("li[data-item-id]")
      
      # find number of visible tweets:
      number_of_tweets = len(tweets)
      
      # keep scrolling:
      driver.execute_script("arguments[0].scrollIntoView();", tweets[-1])
      
      try:
        
        # wait for more tweets to be visible:
        wait.until(wait_for_more_than_n_elements_to_be_present((By.CSS_SELECTOR, "li[data-item-id]"), number_of_tweets))
        
      except TimeoutException:
        
        # if no more are visible the "wait.until" call will timeout. Catch the exception and exit the while loop:
        break
      
      # extract the html for the whole lot:
      page_source = driver.page_source
    
  except TimeoutException:
    
    # if there are no search results then the "wait.until" call in the first "try" statement will never happen and it will time out. So we catch that exception and return no html.
    page_source=None
    
  return page_source

In [ ]:
def extract_tweets(page_source):
  
  soup = bs(page_source,'lxml')
  
  tweets = []
  
  for li in soup.find_all("li", class_='js-stream-item'):
    
    # If our li doesn't have a tweet-id, we skip it as it's not going to be a tweet.
    if 'data-item-id' not in li.attrs:
      continue
    
    else:
      tweet = {
          'tweet_id': li['data-item-id'],
          'text': None,
          'user_id': None,
          'user_screen_name': None,
          'user_name': None,
          'created_at': None,
          'retweets': 0,
          'likes': 0,
          'replies': 0
      }
      
      # Tweet Text
      text_p = li.find("p", class_="tweet-text")
      if text_p is not None:
        tweet['text'] = text_p.get_text()
      
      # Tweet User ID, User Screen Name, User Name
      user_details_div = li.find("div", class_="tweet")
      if user_details_div is not None:
        tweet['user_id'] = user_details_div['data-user-id']
        tweet['user_screen_name'] = user_details_div['data-screen-name']
        tweet['user_name'] = user_details_div['data-name']
      
      # Tweet date
      date_span = li.find("span", class_="_timestamp")
      if date_span is not None:
        tweet['created_at'] = float(date_span['data-time-ms'])
      
      # Tweet Retweets
      retweet_span = li.select("span.ProfileTweet-action--retweet > span.ProfileTweet-actionCount")
      if retweet_span is not None and len(retweet_span) > 0:
        tweet['retweets'] = int(retweet_span[0]['data-tweet-stat-count'])
      
      # Tweet Likes
      like_span = li.select("span.ProfileTweet-action--favorite > span.ProfileTweet-actionCount")
      if like_span is not None and len(like_span) > 0:
        tweet['likes'] = int(like_span[0]['data-tweet-stat-count'])
      
      # Tweet Replies
      reply_span = li.select("span.ProfileTweet-action--reply > span.ProfileTweet-actionCount")
      if reply_span is not None and len(reply_span) > 0:
        tweet['replies'] = int(reply_span[0]['data-tweet-stat-count'])
        
        tweets.append(tweet)
  
  return tweets

In [ ]:
if __name__ == "__main__":

  # start a driver for a web browser:
  driver = init_driver()
  
  # search twitter:
  query = "bitcoin"
  page_source = search_twitter(driver, query)
  
  # extract info from the search results:
  tweets = extract_tweets(page_source)
  
  # ==============================================
  # add in any other functions here
  # maybe some analysis functions
  # maybe a function to write the info to file
  # ==============================================
  
  # close the driver:
  close_driver(driver)

TimeoutException: ignored

---



https://m.blog.naver.com/seodaeho91/221347214246

In [ ]:
startdate=dt.date(year=2022,month=6,day=1) #시작날짜
untildate=dt.date(year=2022,month=6,day=2) # 시작날짜 +1
enddate=dt.date(year=2022,month=6,day=17) # 끝날짜

In [ ]:
query="빅데이터"
totaltweets=[]
while not enddate==startdate:
    url='https://twitter.com/search?q='+query+'%20since%3A'+str(startdate)+'%20until%3A'+str(untildate)+'&src=typed_query'
    driver.get(url)
    html = driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    
    lastHeight = driver.execute_script("return document.body.scrollHeight")
    while True:
        
        dailyfreq={'Date':startdate}

        wordfreq=0
        tweets=soup.find_all("p", {"class": "TweetTextSize"})
                    
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        
        newHeight = driver.execute_script("return document.body.scrollHeight")
        

        if newHeight != lastHeight:
            html = driver.page_source
            soup=BeautifulSoup(html,'html.parser')
            tweets=soup.find_all("p", {"class": "TweetTextSize"})
            wordfreq=len(tweets)
        else:
            dailyfreq['Frequency']=wordfreq
            wordfreq=0
            startdate=untildate
            untildate+=dt.timedelta(days=1)
            dailyfreq={}
            totaltweets.append(tweets)
            break

        lastHeight = newHeight

In [ ]:
df = pd.DataFrame(columns=['Created_Date','Message'])

for i in range(len(totaltweets)):
    for j in range(len(totaltweets[i])):
        df = df.append({'Created_Date': (totaltweets[i][j]).created_at,'Message':(totaltweets[i][j]).text}, ignore_index=True)

In [ ]:
print(totaltweets)

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]




---



https://fouaaa.blogspot.com/2020/06/blog-post.html



---



https://blog.xeros.dev/posts/Python_Selenium_Get_Twitter_FollowerList/

검색은 아니고 팔로워 목록이지만 코드 수정해서 사용 가능할듯